<a href="https://colab.research.google.com/github/sunyeul/ToyProjectLab/blob/feature%2Fnanogpt_tutorial/nanoGPT/nanoGPT_tutorial_gpt2_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nanoGPTのコードレビュー

- https://github.com/karpathy/nanoGPT

![](https://github.com/karpathy/nanoGPT/raw/master/assets/nanogpt.jpg)

In [ ]:
!nvidia-smi

Tue Apr 25 09:07:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 事前設定

In [ ]:
!pip install --upgrade -qq tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.9 MB/s eta 0:00:00


In [ ]:
import math
import tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F

from tqdm.auto import tqdm
from dataclasses import dataclass

torch.manual_seed(3655)

@dataclass
class Config:
    vocab_size: int = 50_304
    batch_size: int = 64
    block_size: int = 256 # what is the maximum context length for predictions?

    train_size: float = 0.8  # valid_sizeは自動で0.2に決まる
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    n_layer: int = 6
    n_head: int = 6
    n_embd: int= 384
    dropout: float = 0.2
    bias: bool = True  # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

enc = tiktoken.get_encoding("gpt2")

## データのダウンロード

In [ ]:
# データセットをダウンロードしましょう。
!wget -N https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-25 03:02:25--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.005s  

Last-modified header missing -- time-stamps turned off.
2023-04-25 03:02:27 (224 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))
print(text[:1_000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [ ]:
# テキストから重複を除いた文字列を取得し、アルファベット順にソートする
chars = sorted(list(set(text)))
print("".join(chars))

# ボキャブラリーのサイズを取得する
vocab_size = len(chars)
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# 文字列をインデックスに変換するための辞書を作成する
# s2iは文字列をインデックスに変換するための辞書
s2i = {ch:i for i, ch in enumerate(chars)}

# インデックスを文字列に変換するための辞書を作成する
# i2sはインデックスを文字列に変換するための辞書
i2s = {i:ch for i, ch in enumerate(chars)}

# 文字列を数値のリストに変換する関数を定義する
encode = lambda s: [s2i[c] for c in s]

# 数値のリストを文字列に変換する関数を定義する
decode = lambda l: ''.join([i2s[i] for i in l])

In [ ]:
# テキストを数値のリストに変換する
data = torch.tensor(enc.encode_ordinary(text), dtype=torch.long)

# データの形状とデータ型を表示する
print(data.shape, data.dtype)

# 先頭の1000文字を表示する
print(data[:1000]) # GPTにとっては、ここで表示される1000文字は以下のようになる

torch.Size([338025]) torch.int64
tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
         3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248,
          461,    11,  2740,    13,   198,   198,  5962, 22307,    25,   198,
         1639,   389,   477, 12939,  2138,   284,  4656,   621,   284,  1145,
          680,    30,   198,   198,  3237,    25,   198,  4965,  5634,    13,
        12939,    13,   198,   198,  5962, 22307,    25,   198,  5962,    11,
          345,   760,   327,  1872,   385,  1526, 28599,   318,  4039,  4472,
          284,   262,   661,    13,   198,   198,  3237,    25,   198,  1135,
          760,   470,    11,   356,   760,   470,    13,   198,   198,  5962,
        22307,    25,   198,  5756,   514,  1494,   683,    11,   290,   356,
         1183,   423, 11676,   379,   674,   898,  2756,    13,   198,  3792,
          470,   257, 15593,    30,   198,   198,  3237,    25,   198,  2949,
          517,  3375,   319,   

## train/validスプリット

In [ ]:
# 学習用データと検証用データに分割する
n = int(Config.train_size * len(data))

train_data = data[:n]
val_data = data[n:]

In [ ]:
# 学習用データを最初のConfig.block_size文字だけに限定する
x = train_data[:Config.block_size]

# 学習用データを2文字目から最初のConfig.block_size+1文字に限定する
y = train_data[1:Config.block_size+1]

# 10回繰り返す
for t in range(10):

    # 入力となる文字列を1文字からt+1文字までに限定する
    context = x[:t+1]

    # 正解の文字を取得する
    target = y[t]

    # 入力がcontextのときに、正解がtargetであることを表示する
    print(f"入力が{context}のときに、正解は{target}です")

入力がtensor([5962])のときに、正解は22307です
入力がtensor([ 5962, 22307])のときに、正解は25です
入力がtensor([ 5962, 22307,    25])のときに、正解は198です
入力がtensor([ 5962, 22307,    25,   198])のときに、正解は8421です
入力がtensor([ 5962, 22307,    25,   198,  8421])のときに、正解は356です
入力がtensor([ 5962, 22307,    25,   198,  8421,   356])のときに、正解は5120です
入力がtensor([ 5962, 22307,    25,   198,  8421,   356,  5120])のときに、正解は597です
入力がtensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597])のときに、正解は2252です
入力がtensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252])のときに、正解は11です
入力がtensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11])のときに、正解は3285です


In [ ]:
def get_batch(split):
    # 入力と正解の小さなバッチを生成する
    # splitが'train'の場合は学習用データから、'val'の場合は検証用データからデータを取得する
    data = train_data if split == 'train' else val_data
    
    # バッチを開始するためのランダムなインデックスを生成する
    idx = torch.randint(high=len(data) - Config.block_size, size=(Config.batch_size,))
    
    # xは、block_sizeの長さのシーケンスのバッチである
    x = torch.stack([data[i:i+Config.block_size] for i in idx])  # [batch_size, block_size]
    
    # yは、xと同じものであるが、1つずつずれている
    y = torch.stack([data[i+1:i+Config.block_size+1] for i in idx])  # [batch_size, block_size]
    
    x, y = x.to(Config.device), y.to(Config.device)
    return x, y

In [ ]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([64, 256])
tensor([[   12,  2435,    11,  ...,   293,   913, 14009],
        [  318,  6004,  1549,  ...,   266,  2326, 14210],
        [ 2767,    25,   198,  ...,  2437,   783,     0],
        ...,
        [  616, 27806,   198,  ...,   198,  2118,   286],
        [12472,   262, 11858,  ...,  8920,   407,   428],
        [26560,   444,   286,  ...,  1175,  2339,   402]], device='cuda:0')
targets:
torch.Size([64, 256])
tensor([[ 2435,    11,   618,  ...,   913, 14009,    11],
        [ 6004,  1549,   517,  ...,  2326, 14210,  4425],
        [   25,   198, 34320,  ...,   783,     0,   508],
        ...,
        [27806,   198,  5211,  ...,  2118,   286,   262],
        [  262, 11858,   339,  ...,   407,   428, 42935],
        [  444,   286, 28064,  ...,  2339,   402, 12968]], device='cuda:0')


## nanoGPTモデルの実装

In [ ]:
def new_gelu(x):
    """
    Google BERT repoに現在実装されているGELU活性化関数の実装（OpenAI GPTと同じ）。
    参考文献：Gaussian Error Linear Units（GELU）論文：https://arxiv.org/abs/1606.08415
    """
    return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))


class LayerNorm(nn.Module):
    """ オプションのバイアスを持つLayerNorm。PyTorchではLayerNormでbias=Falseを簡単にサポートしていない。 """
    
    def __init__(self, ndim, bias):
        super().__init__()
        # 正規化の重みパラメータ
        self.weight = nn.Parameter(torch.ones(ndim))
        # オプションのバイアス
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None
        
    def forward(self, input):
        # Layer Normalizationを計算する
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, eps=1e-5)


class CausalSelfAttention(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # 全てのヘッドに対するkey、query、valueのプロジェクションをバッチ内で実行
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # 出力のプロジェクション
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # 正則化
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # Flash AttentionはGPUを加速するが、PyTorch >= 2.0でのみサポートされる
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # 入力シーケンスの左側にのみ注意が適用されることを保証する因果マスク
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))
    
    def forward(self, x):
        B, T, C = x.size()  # バッチサイズ、シーケンス長、埋め込みサイズ（n_embd）

        # バッチ全体の全てのヘッドのquery、key、valuesを計算し、ヘッドをバッチの先頭に移動
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # [B, n_head, T, head_size]
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # [B, n_head, T, head_size]
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # [B, n_head, T, head_size]
        
        # 因果的自己注意機構; self-attend: [B, n_head, T, head_size] x [B, n_head, head_size, T] -> [B, n_head, T, T]
        if self.flash:
            # Flash Attention CUDAカーネルを使用した効率的な注意
            y = nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout, is_causal=True)
        else:
            # 注意の手動実装
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf"))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v  # [B, n_head, T, T] x [B, n_head, T, head_size] -> [B, n_head, T, head_size]
        y = y.transpose(1, 2).contiguous().view(B, T, C)  # すべてのヘッドの出力を横並びに再構築
        
        # 出力のプロジェクション
        y = self.resid_dropout(self.c_proj(y))
        return y


class MLP(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        # 入力を4倍に拡張する線形層
        self.c_fc = nn.Linear(config.n_embd, config.n_embd * 4, bias=config.bias)
        # 入力サイズに戻す線形層
        self.c_proj = nn.Linear(config.n_embd * 4, config.n_embd, bias=config.bias)
        # GELU非線形関数
        self.act = new_gelu
        # ドロップアウト
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x):
        # GELU非線形関数を適用する
        h = self.act(self.c_fc(x))
        # 元のサイズに戻す
        h2 = self.c_proj(h)
        # 出力をドロップアウト
        return self.dropout(h2)


class Block(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        # Causal Self-Attention
        self.attn = CausalSelfAttention(config)
        # Layer Normalization
        self.ln_1 = LayerNorm(config.n_embd, config.bias)
        # MLP
        self.mlp = MLP(config)
        # Layer Normalization
        self.ln_2 = LayerNorm(config.n_embd, config.bias)
    
    def forward(self, x):
        # Multi-Head Attentionを適用する
        x = x + self.attn(self.ln_1(x))  # 元のコードはこちら
        # x = x + self.ln_1(self.attn(x))
        # MLPを適用する
        x = x + self.mlp(self.ln_2(x))  # 元のコードはこちら
        # x = x + self.ln_2(self.mlp(x))
        return x


class GPT(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config
    
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, config.bias)
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)    
        self.transformer.wte.weight = self.lm_head.weight

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of lengh {t}, block size is {self.config.block_size}"
        
        pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0)  # shape [1, t]

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx)  # token embeddings of shape [b, t, n_embd]
        pos_emb = self.transformer.wpe(pos)  # positional embeddings of shape [1, t, n_embd]
        x = self.transformer.drop(tok_emb + pos_emb)  # [b, t, n_embd]
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)  # [b, t, n_embd]

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x) # [b, t, vocab_size]
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :])  # [b, 1, vocab_size]  note: using list [-1] to preserve the time dim
            loss = None
        
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=0.7, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequencecontext is growing too long, we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)  # [b, min(idx_cond.size(1), block_size), vocab_size]
            # pluck the logits at the final step and scale by desired terperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution to get the next index
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat([idx, idx_next], dim=1)
        
        return idx

## 学習

In [ ]:
# モデルをインスタンス化する
model = GPT(config=Config)
model.to(device=Config.device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50304, 384)
    (wpe): Embedding(256, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=True)
          (c_proj): Linear(in_features=384, out_features=384, bias=True)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_1): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=True)
          (c_proj): Linear(in_features=1536, out_features=384, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=384, out_features=50304, bias=False)
)

In [ ]:
# 学習可能なパラメーター数を計算
num_params = 0

for parameter in model.parameters():
    num_params += len(parameter)

print(num_params)

102016


In [ ]:
# 順伝播を実行し、ログオッズと損失を取得する
logits, loss = model(xb, yb)

# logitsの形状と損失を表示する
print(logits.shape)
print(loss)

# モデルによって生成されたテキストを表示する
generated_text = model.generate(idx=torch.zeros((1, 1,), dtype=torch.long, device=Config.device), max_new_tokens=100)[0].tolist()
decoded_text = enc.decode(generated_text)
print(decoded_text)

torch.Size([64, 256, 50304])
tensor(10.8837, device='cuda:0', grad_fn=<NllLossBackward0>)
!uffleScale researched:,Dragon netted Britann Parfrac catches destruction的 masse Allan impulse cupglgan Kil email creations Present194dev redeemed productivity servedinventory 237SathodoxpmwikiAdjust $\verning � HorseRF Adren strands wid opt cannabin scored provoke Heard Dy >> text decimalpson disturook concernathered freshwater Chic Conn march vibrations Conscious remake Sly nightclub bu� �tick 374 commonly soaked cannabinoid variablesgc Freud verified---------iamondFake Gob politElsewhereisan 252 metric TransformationenegFN likewise Heritage Chicago Casino poisonedidy jealouszu COL champions Costume a


In [ ]:
# PyTorchのオプティマイザを作成する
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# 10,000回のステップを実行する
EPOCH = 10_000
for steps in tqdm(range(EPOCH)):
    
    # データのバッチをサンプリングする
    xb, yb = get_batch('train')

    # 勾配をゼロに設定
    optimizer.zero_grad(set_to_none=True)

    # 損失を評価する
    logits, loss = model(xb, yb)

    # 逆伝播を計算する
    loss.backward()

    # パラメータを更新する
    optimizer.step()

# 損失を表示する
print(loss.item())

  0%|          | 0/10000 [00:00<?, ?it/s]

0.1752912998199463


In [ ]:
# Additional information
PATH = "model.pth"

torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item(),
            }, PATH)

In [ ]:
# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

## 文章生成

In [ ]:
# モデルによって生成されたテキストを表示する
generated_text = model.generate(idx=torch.zeros((1, 1), dtype=torch.long, device=Config.device), max_new_tokens=500)[0].tolist()

decoded_text = enc.decode(generated_text)
print(decoded_text)

!

BUCKINGHAM:
You are, my lord: would unto this holy mother
Will our father, for our king is well,
The queen's son, and the duke
Have taken sanctuary.

GLOUCESTER:
The thought upon your royal cousin, gentle lady king,
You love not me, nor, nor none
But such demand thought: you, my heart have I mean,
I was not sanctuary.

BUCKINGHAM:
My lord, I have consider'd in my mind
The book that you did make you for the time
To soften of her false presence.

LADY ANNE:
The manner is dead, perhaps
Upon my kindred you know the shoulders.

GLOUCESTER:
Do, thou liest like an Edward,
To be revenged on the entreaties, he that you come?

BUCKINGHAM:
To for ever, my lord I have.

KING RICHARD III:
Why, madam, thou art.

CATESBY:
I hope so swear, my lord.

KING RICHARD III:
Dothst thou resolve to kill a friend of mine. Hark, sir, ere you be crown'd his mind?

QUEEN ELIZABETH:
You have, and he holds me wrongfully;
And I shall be unadvisedly to my oath.

QUEEN ELIZABETH:
But, I'll say my lord.

GLOUCESTER:


In [ ]:
# モデルによって生成されたテキストを表示する

prompt = "Hello, "
generated_text = model.generate(idx=torch.tensor([enc.encode(prompt)], device=Config.device), max_new_tokens=500)[0].tolist()

decoded_text = enc.decode(generated_text)
print(decoded_text)

Hello,  Well,
'Twill be my purpose.
Were it you the party in this peace?

AUFIDIUS:
I do, I'll tell:
And follow it.

MICINIUS:
A hundred drops. I will seal upon indeed,
And yet. Aufidius, ho!

LARTIUS:
This is present, my country,
And when e'er man shall meet the peace.

MARCIUS:
I do not know this walls:
Lie, be gone.

AUFIDIUS:
The second patricians shall tell: I cannot tell,
'Tis such a hayard.
Beat thou tellestest,
And see this sword to see me, that it speak of breath
Lest child or I do do lie, behold him:
Trail all my days,
That ere I revengeful eyes over it,
Slening, or some hour in this present,
Hath more cause to exile to banishment.

CORIOLANUS:
Give me thy hand: Come.

AUFRIAR LAURENCE:
Hatho my down;
Or manage it gives me word,
O, I'll help thy bones:
Haply he is within, and my lady,
His help up like a hot behold,
And cruel death in my breath
Which laid me to sun's eyes.

ROMEO:
Then move me not;I have no such a feeling
To me, if I can do so much before:
Be thou overheard me

## おまけ

### 事前学習済みモデル使用

In [ ]:
!pip install --upgrade transformers tiktoken -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.4 MB/s eta 0:00:00


In [ ]:
@classmethod
def from_pretrained(cls, model_type, override_args=None):
    assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
    override_args = override_args or {} # default to empty dict
    # override_argsがdropoutだけであることを確認する
    assert all(k == 'dropout' for k in override_args)
    # GPT2LMHeadModelを使用する
    from transformers import GPT2LMHeadModel
    print("loading weights from pretrained gpt: %s" % model_type)

    # n_layer, n_head, n_embdの値はmodel_typeによって決定される
    config_args = {
        'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
        'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
        'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
        'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
    }[model_type]
    # vocab_size, block_size, biasの値はGPTモデルのチェックポイントによって常に同じ値を取る
    print("forcing vocab_size=50257, block_size=1024, bias=True")
    config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
    config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
    config_args['bias'] = True # always True for GPT model checkpoints
    # dropout rateを上書きする場合
    if 'dropout' in override_args:
        print(f"overriding dropout rate to {override_args['dropout']}")
        config_args['dropout'] = override_args['dropout']
    # からのminGPTモデルを初期化する
    config = Config(**config_args)
    model = GPT(config=config)
    sd = model.state_dict()
    sd_keys = sd.keys()
    sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # mask / bufferは削除

    # HuggingFace/Transformersモデルを初期化する
    model_hf = GPT2LMHeadModel.from_pretrained(model_type)
    sd_hf = model_hf.state_dict()

    # 全てのパラメータが名前と形状が一致するように、名前と形状を合わせてコピーする
    sd_keys_hf = sd_hf.keys()
    sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # これらは無視する
    sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # 同上
    transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

    # OpenAIのチェックポイントでは"Conv1D"モジュールを使用していますが、
    # vanilla Linearを使用するように変換する必要があります。
    # そのため、重みを転置する必要があります。
    assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
    for k in sd_keys_hf:
        if any(k.endswith(w) for w in transposed):
            # Conv1Dの重みに対する特別な処理が必要です。
            assert sd_hf[k].shape[::-1] == sd[k].shape
            with torch.no_grad():
                sd[k].copy_(sd_hf[k].t())
        else:
            # その他のパラメータを単純にコピーします。
            assert sd_hf[k].shape == sd[k].shape
            with torch.no_grad():
                sd[k].copy_(sd_hf[k])

    return model

GPT.from_pretrained = from_pretrained

In [ ]:
model = GPT(config=Config).from_pretrained(model_type='gpt2')
model.to(device=Config.device)

loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_1): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")

In [ ]:
prompt = 'A person with a high school education gets sent back into the 1600s and tries to explain science and technology to the people. [endprompt]'

generated_text = model.generate(
    idx=torch.tensor([enc.encode_ordinary(prompt)], device=Config.device),
    max_new_tokens=500,
    )[0].tolist()

decoded_text = enc.decode(generated_text)

print(decoded_text)

A person with a high school education gets sent back into the 1600s and tries to explain science and technology to the people. [endprompt]

The three most important thing is to have a school and we had a little more important thing than we have a school has an idea I- [end]
But we have a school idea idea and a school idea and we have a school idea idea in the school idea and we have any pair of ideas you know we have any school idea ideas we have some school idea idea idea about school idea we have no we have the idea and we have the idea we have a school idea we have a school idea we have a school idea idea that we have no we have the idea we have not a school idea we have a school idea we have a school idea we have a school idea we have a school idea we have a school idea we have a school we have a school idea one that a school school idea we have a school idea we have a school idea idea we have a school idea we have a school idea we have a school we have a school idea we have a scho

### Fine-Tuning

In [ ]:
# PyTorchのオプティマイザを作成する
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# 100,000回のステップを実行する
for steps in tqdm(range(1_000)):
    
    # データのバッチをサンプリングする
    xb, yb = get_batch('train')
    xb = "".join(["".join([i2s[x.tolist()] for x in xx]) for xx in xb])
    yb = "".join(["".join([i2s[y.tolist()] for y in yy]) for yy in yb])

    xb = torch.tensor([enc.encode_ordinary(x) for x in xb], device=Config.device)
    yb = torch.tensor([enc.encode_ordinary(y) for y in yb], device=Config.device)

    # 損失を評価する
    logits, loss = model(xb, yb)

    # 勾配をゼロに設定して、逆伝播を計算する
    optimizer.zero_grad(set_to_none=True)
    loss.backward()

    # パラメータを更新する
    optimizer.step()

# 損失を表示する
print(loss.item())

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
# モデルによって生成されたテキストを表示する
generated_text = model.generate(idx=torch.zeros((1, 1), dtype=torch.long, device=Config.device), max_new_tokens=500)[0].tolist()

# prompt = "Hello, "
# generated_text = model.generate(idx=torch.tensor([encode(prompt)], device=Config.device), max_new_tokens=500)[0].tolist()

decoded_text = decode(generated_text)
print(decoded_text)